# <span style="color:seagreen; font-family:freestyle script; font-size:4em">LNU till relationsdatabas</span> <br><span style="color:black; font-family:freestyle script; font-size:3em">Frekvenstabeller från LNU:s kodbok</span>
***

In [1]:
import pandas as pd
import camelot
import re
import numpy as np

In [2]:
tables = camelot.read_pdf('LNU41.pdf')

if tables.n == 0:
    print('INGEN tabell hehe')
else:
    df = tables[0].df

In [11]:
rutor[2]

['Z19 ',
 'ISCO88 EGEN ',
 'NY ',
 'Eget  yrke  enligt  The  International  Standard  Classification  of  Occupations,  1988  års  indelning  (ISCO88). ',
 'Redovisas ej']

**TEST**

In [20]:
len([0,1,2])

3

In [23]:
# reg1: sträng börjar med en siffra eller "<".
reg1 = re.compile(r'^\d|<')

reg11 = re.compile(r'\d.$')

# reg2: sträng börjar med någonting annat än siffra.
reg2 = re.compile(r'^\D')

# reg3: variabelnamn.
reg3 = re.compile(r'^Z\d|^PZ\d')

# reg4: sträng är en siffra med en minst en decimal.
reg4 = re.compile(r'\d,|\d\.')

# reg5: sträng innehåller bokstäver.
reg5 = re.compile(r'[a-zA-Z]')
                  
rutor = [df.loc[i][0].split('\n') for i in df.index]

delar = []

for i in range(32, 357):
    # Läs in sidan i från LNU:s kodbok
    tables = camelot.read_pdf('LNU{}.pdf'.format(i))
    
    print(i)
    # Gör om "tabellen" på sidan till en DataFrame. Vissa sidor, t.ex. 36, har ingen tabell alls. 
    # Kolla därför att tabell finns.
    if tables.n == 0:
        # Om tabell "inte finns" skapas en df med en rad som säger sidbrytning i alla kolumner.
        A = pd.DataFrame(data = [['sidbrytning', 'sidbrytning', 'sidbrytning', 'sidbrytning']],
                 columns = ['Kod', 'Svarstext', 'Frekvens', 'Variabel'])
        # Denna df läggs sedan till i listan delar.
        delar.append(A)
    else:
        df = tables[0].df
        rutor = [df.loc[i][0].split('\n') for i in df.index]
        

        # j är en ruta i kodboken.
        for j in range(len(rutor)):
            #print('j:' + str(j))

            namn = []
            koder = []
            texter = []
            frekvenser = []

            # k är en sträng i en ruta.
            for k in range(len(rutor[j])):
                #print('k:' + str(k))

                kod = rutor[j][k]

                # När jag når 'Totalt ' (eller 'Total ') är kodlistan slut. Notera whitespace i slutet av strängen...
                if kod[:4] == 'Tota':
                    break  
                # Vid sidbrytningar finns inget 'Totalt ' och därför hamnar k+2 utanför index range.
                # Ifall variabeln saknar frekvenstabell och hamnar i sidbrytning (Z112) blir även k+1 out of range.
                if (k == len(rutor[j]) - 2) or (len(rutor[j]) < 3):
                    break

                # Går jag förbi 'Totalt ' kommer jag få felmeddelande att k+2 ligger utanför index.
                # Därför kollar jag detta innan text och frekvens skapas.
                text = rutor[j][k+1]
                frekvens = rutor[j][k+2]

                # Dessa villkor blir så långa. Det blir lättare att läsa förkortningar.
                # t1: sträng börjar med siffra eller "<"
                t1 = bool(reg1.findall(kod))
                
                # t11: sträng slutar med en siffra (typ... se reg11 för detaljer)
                t11 = bool(reg11.findall(kod))

                # t2: Strängen innehåller en siffra som följs av en punkt.
                t2 = bool(reg4.findall(rutor[j][k]))

                # t2: Nästa sträng innehåller en siffra som följs av en punkt.
                t3 = bool(reg4.findall(rutor[j][k+1]))

                # Den första sträng som fångas av reg3 måste vara variabelnamnet.
                if (bool(reg3.findall(kod)) == True) & (len(namn) == 0):
                    namn.append(kod)

                # En kod måste inledas med en siffra (eller "<") (t1) samt samt sluta med en siffra (t11).
                if (t1 == True) & (t11 == True):
                    # Ifall detta är den första koden
                    if (len(koder) == 0):
                        #print('första koden')

                        # Ifall text börjar med whitespace eller bokstav så lägg till i listorna som vanligt.
                        if bool(reg2.findall(text)) == True:
                            koder.append(kod)
                            texter.append(text)
                            frekvenser.append(frekvens)
                        # Ifall text börjar med siffra är det snarare än en frekvens.
                        else:
                            koder.append(kod)
                            texter.append(' ')
                            frekvenser.append(text)

                    # Ifall listan med koder inte är tom
                    # Kolla så att k-1 (ifall k är en frekvens) och k-2 (ifall k är en andel) varken är text eller whitespace.
                    elif (t2 == False) & (t3 == False): 
                        #print('inte första koden')
                        if bool(reg2.findall(text)) == True:
                            koder.append(kod)
                            texter.append(text)
                            frekvenser.append(frekvens)
                        else:
                            koder.append(kod)
                            texter.append(' ')
                            frekvenser.append(text)
                            
            # Variabler som Z18 har ingen frekvenstabell. Lägg då in 'saknas' i dataframen
            if len(koder) == 0:
                koder.append('saknas')
                texter.append('saknas')
                frekvenser.append('saknas')    
                    
                #print('koder: ' + str(koder), 'texter: ' + str(texter), 'frekvenser: ' + str(frekvenser), sep = '\n')
            if len(namn) == 0:
                namn.append('sidbrytning')
            #print('namn: ' + namn[0])

            # Innan nästa ruta i kodboken (j) så skapas en df med koder, texter och frekvenser som kolumner.
            A = pd.DataFrame.from_dict({'Kod': koder, 'Svarstext': texter, 'Frekvens': frekvenser})

            # Lägg till en kolumn för variabelnamn.
            A['Variabel'] = namn[0]

            #Lägg till den färdiga frekvenstabellen i listan "delar".
            delar.append(A)

32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298


In [24]:
LNU = pd.concat(delar)
LNU

,Kod,Svarstext,Frekvens,Variabel
0,1,Besöksintervju,3278,Z8
1,2,Telefonintervju,1137,Z8
0,1,Intervjun gjord med ip,4383,Z9
1,2,Intervjun gjord med annan än ip,32,Z9
0,1,Intervjun gjord på svenska,4348,Z10
...,...,...,...,...
2,3,Yrkesinriktat gymnasium 2-3 år,806,PZ110
3,4,Teoretiskt gymnasium 3 år,166,PZ110
4,5,Postgymnasium,542,PZ110
5,6,Universitet- eller högskoleexamen,420,PZ110


**TEST IGEN**

In [18]:
tables = camelot.read_pdf('LNU71.pdf')

if tables.n == 0:
    print('INGEN tabell hehe')
else:
    df = tables[0].df
    
rutor = [df.loc[i][0].split('\n') for i in df.index]